<a href="https://colab.research.google.com/github/inhyek/machine_learning_toddler/blob/master/%EC%8B%A4%EC%8A%B5(Dacon_Funda).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 문제
2년 전 부터 2019년 2월 28일까지의 카드 거래 데이터를 이용해 2019-03-01부터 2019-05-31까지의 각 상점별 3개월 총 매출을 예측하는 것입니다.

 

### 데이터 설명 

 

  [Files]

 

  funda_train.csv - 2019-02-28일까지의 상점별 카드 매출 내역

  submission.csv – 제출 파일의 형식

 

  [Data fields]

  funda_train.csv

  1. store_id : 상점의 고유 아이디

  2. card_id : 사용한 카드의 고유 아이디

  3. card_company : 비식별화된 카드 회사

  4. trasacted_date : 거래 날짜

  5. transacted_time : 거래 시간( 시:분 )

  6. installment_term : 할부 개월 수( 포인트 사용 시 (60개월 + 실제할부개월)을 할부개월수에 기재한다. )

  7. region : 상점의 지역

  8. type_of_business : 상점의 업종

  9. amount : 거래액(단위는 원이 아닙니다)

 

  submission.csv

  1. store_id : 상점의 고유 아이디

  2. amount : 사용자들은 상점별로 2019-03-01부터 2019-05-31까지의 총 매출을 예측하여 기입

## Load Dataset
데이터를 불러옵니다.

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
!ls /gdrive/My\ Drive/Colab\ Notebooks/data

diabetes.csv		 funda_train.csv   titanic_train.csv
funda_sampled_data.xlsx  titanic_test.csv


In [0]:
import pandas as pd

In [0]:
# 판다스의 read_csv로 funda_train.csv 파일을 읽어옵니다.
# 여기서 transacted_date은 특별히 날짜로 해석하기 위해 parse_dates 옵션에 넣어줍니다.
# 읽어온 데이터를 train이라는 이름의 변수에 할당합니다
train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data/funda_train.csv', parse_dates =['transacted_date'])

In [0]:
# 데이터타입 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 9 columns):
store_id            int64
card_id             int64
card_company        object
transacted_date     datetime64[ns]
transacted_time     object
installment_term    int64
region              object
type_of_business    object
amount              float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 450.2+ MB


In [0]:
# drop
train = train.drop(['card_id','card_company',  'transacted_time','installment_term', 'region', 'type_of_business', 'installment_term'], axis = 1)

In [0]:
train.shape

train.isnull().sum()

store_id           0
transacted_date    0
amount             0
dtype: int64

In [0]:
# 상위 3개 데이터 확인 
train.head(3)

,store_id,transacted_date,amount
0,0,2016-06-01,1857.142857
1,0,2016-06-01,857.142857
2,0,2016-06-01,2000.000000


In [0]:
# 상점 개수 확인
train['store_id'].value_counts()

1330    9518
1196    9471
1171    9391
710     9347
826     9328
15      9306
1400    9246
2076    9226
831     9217
2027    9138
757     9125
646     9102
1218    9095
1046    9087
942     9042
2009    9030
821     9029
1970    9014
1155    8997
553     8990
2136    8972
2125    8967
1170    8966
1387    8943
1466    8940
2036    8933
311     8926
1141    8924
197     8908
676     8903
        ... 
1881     547
1784     547
488      547
576      546
789      546
1435     543
982      539
1524     539
1929     536
248      535
334      533
105      533
808      533
1723     527
1853     526
2014     526
1795     524
722      522
250      519
412      501
237      498
788      492
1204     479
993      462
448      456
1974     429
1240     426
795      231
2119     112
1063      72
Name: store_id, Length: 1967, dtype: int64

In [0]:
#기본적인 기술통계확인 --> 특히 중요한것. 매출액 min, max, mean, median --> 확인해보니 별 도움이 안되는 것 같다.
train.describe()

,store_id,amount
count,6.556613e+06,6.556613e+06
mean,1.084930e+03,1.043511e+04
std,6.152183e+02,3.104031e+04
min,0.000000e+00,-2.771429e+06
25%,5.860000e+02,2.142857e+03
50%,1.074000e+03,4.285714e+03
75%,1.615000e+03,8.571429e+03
max,2.136000e+03,5.571429e+06


### 보류

In [0]:
import datetime

train['datetime-date']= train["transacted_date"].dt.date
train["datetime-year"] = train["transacted_date"].dt.year
train["datetime-month"] = train["transacted_date"].dt.month
train["datetime-day"] = train["transacted_date"].dt.day
train["datetime-dayofweek"] = train["transacted_date"].dt.dayofweek


train.head(3)

,store_id,transacted_date,amount,datetime-date,datetime-year,datetime-month,datetime-day,datetime-dayofweek
0,0,2016-06-01,1857.142857,2016-06-01,2016,6,1,2
1,0,2016-06-01,857.142857,2016-06-01,2016,6,1,2
2,0,2016-06-01,2000.000000,2016-06-01,2016,6,1,2


In [0]:
train = train.drop(['transacted_date','datetime-date','datetime-year','datetime-day'], axis=1)

KeyError: ignored

In [0]:
train.loc[train["datetime-dayofweek"] == 0, "datetime-dayofweek(humanized)"] = "Monday"
train.loc[train["datetime-dayofweek"] == 1, "datetime-dayofweek(humanized)"] = "Tuesday"
train.loc[train["datetime-dayofweek"] == 2, "datetime-dayofweek(humanized)"] = "Wednesday"
train.loc[train["datetime-dayofweek"] == 3, "datetime-dayofweek(humanized)"] = "Thursday"
train.loc[train["datetime-dayofweek"] == 4, "datetime-dayofweek(humanized)"] = "Friday"
train.loc[train["datetime-dayofweek"] == 5, "datetime-dayofweek(humanized)"] = "Saturday"
train.loc[train["datetime-dayofweek"] == 6, "datetime-dayofweek(humanized)"] = "Sunday"

In [0]:
train.head()

,store_id,amount,datetime-month,datetime-dayofweek,datetime-dayofweek(humanized)
0,0,1857.142857,6,2,Wednesday
1,0,857.142857,6,2,Wednesday
2,0,2000.000000,6,2,Wednesday
3,0,7857.142857,6,2,Wednesday
4,0,2000.000000,6,3,Thursday


## Preprocessing
데이터를 편하게 분석하고, 머신러닝 알고리즘에 집어넣기 위해 전처리 작업을 진행합니다. 

### Parse Datetime
날짜(datetime) 컬럼을 연, 월, 일, 시, 분, 초로 따로 나누겠습니다.
여섯개의 컬럼이 추가됩니다.

##### 결측치를 확인해봅니다.

In [0]:
train.isna().sum()

store_id                         0
amount                           0
datetime-month                   0
datetime-dayofweek               0
datetime-dayofweek(humanized)    0
dtype: int64

In [0]:
print("Original features:\n", list(train.columns), "\n")
train = pd.get_dummies(train)
print("겟더미 후 features: \n", list(train.columns))
train.head()

Original features:
 ['store_id', 'amount', 'datetime-month', 'datetime-dayofweek', 'datetime-dayofweek(humanized)'] 

겟더미 후 features: 
 ['store_id', 'amount', 'datetime-month', 'datetime-dayofweek', 'datetime-dayofweek(humanized)_Friday', 'datetime-dayofweek(humanized)_Monday', 'datetime-dayofweek(humanized)_Saturday', 'datetime-dayofweek(humanized)_Sunday', 'datetime-dayofweek(humanized)_Thursday', 'datetime-dayofweek(humanized)_Tuesday', 'datetime-dayofweek(humanized)_Wednesday']


,store_id,amount,datetime-month,datetime-dayofweek,datetime-dayofweek(humanized)_Friday,datetime-dayofweek(humanized)_Monday,datetime-dayofweek(humanized)_Saturday,datetime-dayofweek(humanized)_Sunday,datetime-dayofweek(humanized)_Thursday,datetime-dayofweek(humanized)_Tuesday,datetime-dayofweek(humanized)_Wednesday
0,0,1857.142857,6,2,0,0,0,0,0,0,1
1,0,857.142857,6,2,0,0,0,0,0,0,1
2,0,2000.000000,6,2,0,0,0,0,0,0,1
3,0,7857.142857,6,2,0,0,0,0,0,0,1
4,0,2000.000000,6,3,0,0,0,0,1,0,0


In [0]:
train.drop(['datetime-dayofweek'], axis=1)

,store_id,amount,datetime-month,datetime-dayofweek(humanized)_Friday,datetime-dayofweek(humanized)_Monday,datetime-dayofweek(humanized)_Saturday,datetime-dayofweek(humanized)_Sunday,datetime-dayofweek(humanized)_Thursday,datetime-dayofweek(humanized)_Tuesday,datetime-dayofweek(humanized)_Wednesday
0,0,1857.142857,6,0,0,0,0,0,0,1
1,0,857.142857,6,0,0,0,0,0,0,1
2,0,2000.000000,6,0,0,0,0,0,0,1
3,0,7857.142857,6,0,0,0,0,0,0,1
4,0,2000.000000,6,0,0,0,0,1,0,0
5,0,2000.000000,6,0,0,0,0,1,0,0
6,0,2000.000000,6,0,0,0,0,1,0,0
7,0,7857.142857,6,0,0,0,0,1,0,0
8,0,2000.000000,6,0,0,0,0,1,0,0
9,0,1857.142857,6,0,0,0,0,1,0,0


In [0]:
train.sort_values(by=['store_id'])

,store_id,amount,datetime-month,datetime-dayofweek,datetime-dayofweek(humanized)_Friday,datetime-dayofweek(humanized)_Monday,datetime-dayofweek(humanized)_Saturday,datetime-dayofweek(humanized)_Sunday,datetime-dayofweek(humanized)_Thursday,datetime-dayofweek(humanized)_Tuesday,datetime-dayofweek(humanized)_Wednesday
0,0,1857.142857,6,2,0,0,0,0,0,0,1
2978,0,8571.428571,3,3,0,0,0,0,1,0,0
2979,0,8571.428571,3,4,1,0,0,0,0,0,0
2980,0,2000.000000,3,4,1,0,0,0,0,0,0
2981,0,5714.285714,3,4,1,0,0,0,0,0,0
2982,0,8571.428571,3,4,1,0,0,0,0,0,0
2983,0,8571.428571,3,4,1,0,0,0,0,0,0
2984,0,14285.714286,3,5,0,0,1,0,0,0,0
2985,0,27857.142857,3,5,0,0,1,0,0,0,0
2986,0,1857.142857,3,6,0,0,0,1,0,0,0


In [0]:
train.columns

Index(['store_id', 'amount', 'datetime-month', 'datetime-dayofweek',
       'datetime-dayofweek(humanized)_Friday',
       'datetime-dayofweek(humanized)_Monday',
       'datetime-dayofweek(humanized)_Saturday',
       'datetime-dayofweek(humanized)_Sunday',
       'datetime-dayofweek(humanized)_Thursday',
       'datetime-dayofweek(humanized)_Tuesday',
       'datetime-dayofweek(humanized)_Wednesday'],
      dtype='object')

In [0]:
store_id_417 = train[train.store_id ==417]
store_id_417.shape

(7695, 11)

In [0]:
store_id_1 = train[train.store_id == 1]
store_id_1.shape

(3782, 11)

In [0]:
store_id_1330 = train[train.store_id == 1330]
store_id_1330.shape

(9518, 11)

In [0]:
store_id_1196 = train[train.store_id == 1196]
store_id_1196.shape

(9471, 11)

In [0]:
feature_names = ['datetime-month', 'datetime-dayofweek',
       'datetime-dayofweek(humanized)_Friday',
       'datetime-dayofweek(humanized)_Monday',
       'datetime-dayofweek(humanized)_Saturday',
       'datetime-dayofweek(humanized)_Sunday',
       'datetime-dayofweek(humanized)_Thursday',
       'datetime-dayofweek(humanized)_Tuesday',
       'datetime-dayofweek(humanized)_Wednesday']
label_name = ['amount']

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(store_id_417[feature_names], store_id_417[label_name], test_size=0.25, random_state=27)

In [0]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=27)
tree.fit(X_train, y_train)
print("훈련 세트 정확도: {:.3f}".format(tree.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(tree.score(X_test, y_test)))

훈련 세트 정확도: 0.014
테스트 세트 정확도: -0.006


In [0]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=5)
knn.fit(X_train, y_train)
print(knn.score(X_train,y_train))
print(knn.score(X_test,y_test))

AttributeError: ignored

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse=np.sqrt(mse)
print(mse, rmse)
print(r2_score(y_test,y_preds))

34112988.4762059 5840.632540761822
-0.001842690374867395
